# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pyquation import Equation, Variable
from sympy import *
from sympy.physics.units import *
from sympy import S

# The Basics

In [3]:
F = Variable(force, newtons, description='May it be with you.')
M = Variable(mass, kilogram)
A = Variable(acceleration, meters/second**2, ('A', 'accel'))
v = Variable(velocity, meters/second, 'v')
r = Variable(length, meters, 'r', name='radius')

In [4]:
assert F.name == 'F'
assert F.full_name == 'force'
assert F.all_names == ('F', 'force')
assert A.name == 'A'
assert A.all_names == ('A', 'acceleration', 'accel')

In [5]:
nlaw = Equation(F, M*A, 'This is Newtons 2nd law of motion')

# 1 less var
Equation.show_solved_term = False
assert nlaw(F=4*newton, M=2*kilogram) == 2*meters/second**2

# show_solved_term
Equation.show_solved_term = True
assert nlaw(F=4*newton, M=2*kilogram) == Eq(A, 2*meters/second**2)
Equation.show_solved_term = False
assert nlaw(F=4*newton, M=2*kilogram, show_solved_term=True) == Eq(A, 2*meters/second**2)
Equation.show_solved_term = True
assert nlaw(F=4*newton, M=2*kilogram, show_solved_term=False) == 2*meters/second**2
Equation.show_solved_term = False

# convert_to(solve(nlaw(F=4*newton, M=7*pound), dict=True)[0][A], A.default_unit)

# same number of vars
assert nlaw(F=4*newton, M=2*kilogram, A=2*meters/second**2)
assert not nlaw(F=4*newton, M=2*kilogram, A=3*meters/second**2)

# Custom units works
assert nlaw(F=4*newton, M=2*kilogram, units=feet/second**2) == S(2500)/381 * feet/second**2

# Too few vars works
assert nlaw(F=4*newton) == Equation(4*newton, A*M)
assert nlaw(M=2*kilogram, units=gram) == Equation(F, 2000*A*gram)

In [6]:
nlaw(A=9.81*meters/second**2, M=12*kg)

117.72*newton

In [7]:
assert nlaw.variables == {A, F, M}

In [8]:
assert nlaw.restructure(M) == [Equation(M, F/A)]

# Naming, making sure things look right

In [9]:
nlaw.help()

Eq(F, A*M)

where:
	A is acceleration in units of meter/second**2
	M is mass in units of kilogram
	F is force in units of newton
		May it be with you.

This is Newtons 2nd law of motion


In [10]:
nlaw.full_form()

Eq(F, A*M)

In [11]:
nlaw()

Eq(F, A*M)

In [12]:
# Parameter names
assert nlaw(M=3*kg, F=2*newton) == nlaw(mass=3*kg, force=2*newton)
assert (
    nlaw(M=2*kg, F=2*newton, F_unc=1, M_unc=2) ==
    nlaw(mass=2*kg, F=2*newton, force_unc=1, mass_unc=2) ==
    nlaw(mass=2*kg, force=2*newton, unc_force=1, unc_M=2) ==
    nlaw(mass=2*kg, F=2*newton, δforce=1, δM=2)
)

# Uncertianties

In [13]:
# uncertinties with too few vars dont do anything
assert nlaw(mass=3*kg, M_unc=2, δforce=3) == nlaw(mass=3*kg)

In [14]:
assert nlaw(A=2*meters/second**2, F=4*newtons, F_unc=1, A_unc=1, units=pound)[1] == 50000000*sqrt(5)/45359237
assert nlaw(A=2*meters/second**2, F=4*newtons, F_unc=1, A_unc=1)[1] == sqrt(5)/2

# The remove_units function

In [15]:
from functools import reduce
from operator import mul
from sympy import flatten

# This is copied from Equation.py exactly
def remove_units(expr):
    return reduce(mul, (i for i in flatten(expr.as_coeff_mul()) if not len(i.atoms(Unit))))

In [16]:
e = convert_to(3*foot, meters)
assert remove_units(e) == S(1143)/1250
assert 4 == remove_units(4*newtons)
assert 4.3 == remove_units(4.3*meters/second**2)

# CombinedEquation

In [17]:
centripetal_motion = Equation(A, (v**2)/r)

In [18]:
radius_of_a_centrifuge = nlaw + centripetal_motion
# We could do something here like rename variables (F to "force_to_exert" or something) and set variables equal to each other
radius_of_a_centrifuge(M=3*kg, F=12*newton, v=10*meters/second)
# Theoretically, this would return a dict of {Variables: values} for all the variables in all the equations

{M: 3*kilogram,
 F: 12*newton,
 v: 10*meter/second,
 A: 4*meter/second**2,
 r: 25*meter}

In [27]:
assert radius_of_a_centrifuge(M=3*kg, F=12*newton, v=10*meters/second) == {
    M: 3*kilogram,
    F: 12*newton,
    v: 10*meter/second,
    A: 4*meter/second**2,
    r: 25*meter
}
# assert radius_of_a_centrifuge(M=3*kg, F=12*newton) == {M: 3*kilogram, F: 12*newton, A: 4*meter/second**2, v: v, r: r}
assert radius_of_a_centrifuge(M=3*kg, F=12*newton, fill_unknowns=None) == {M: 3*kilogram, F: 12*newton, A: 4*meter/second**2, v: None, r: None}
assert radius_of_a_centrifuge(M=3*kg, F=12*newton, fill_unknowns=False) == {M: 3*kilogram, F: 12*newton, A: 4*meter/second**2}

In [55]:
assert radius_of_a_centrifuge(M=3*kg, F=12*newton, unc_F=1, unc_M=1, fill_unknowns=False) == {
    M: 3*kilogram,
    F: 12*newton,
    A: 4*meter/second**2,
    Symbol('δF', positive=True): 1,
    Symbol('δM', positive=True): 1,
    Symbol('δA', positive=True): sqrt(17)/3
}

assert radius_of_a_centrifuge(M=3*kg, F=12*newton, unc_F=1, unc_M=1, fill_unknowns=True) == {
    M: 3*kilogram,
    F: 12*newton,
    A: 4*meter/second**2,
    r: Symbol('r'),
    v: Symbol('v'),
    Symbol('δF', positive=True): 1,
    Symbol('δM', positive=True): 1,
    Symbol('δA', positive=True): sqrt(17)/3,
    Symbol('δr', positive=True): Symbol('δr', positive=True),
    Symbol('δv', positive=True): Symbol('δv', positive=True)
}

assert radius_of_a_centrifuge(M=3*kg, F=12*newton, unc_F=1, M_unc=1, fill_unknowns=None) == {
    M: 3*kilogram,
    F: 12*newton,
    A: 4*meter/second**2,
    r: None,
    v: None,
    Symbol('δF', positive=True): 1,
    Symbol('δM', positive=True): 1,
    Symbol('δA', positive=True): sqrt(17)/3,
    Symbol('δr', positive=True): None,
    Symbol('δv', positive=True): None
}